In [1]:
from jupyter_dash import JupyterDash

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import dash_bootstrap_components as dbc
import dash_html_components as html
from datetime import datetime as dt

In [85]:
# Since we're adding callbacks to elements that don't exist in the app.layout,
# Dash will raise an exception to warn us that we might be
# doing something wrong.
# In this case, we're adding the elements through a callback, so we can ignore
# the exception.

app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

df = pd.read_json('C:/Users/Utilisateur/Desktop/Cours/M1/projet inter-promo/groupe 7/donnees/df_final.json')

df = df.dropna(subset=['art_title', 'art_url','art_content','src_img','art_tag']).drop(['art_id'], axis=1).reset_index(drop=True)


tags = traitement_themes()
dic = {}
for i in df['art_tag']:
    if i is None :
        continue
    else :
        for j in tags:
            if j in i:
                if j in dic:
                    dic[j] += 1
                else :
                    dic[j] = 1

top_tags = sorted(dic.items(),key = lambda t:t[1])[-12:-2]

tags_index = 11

# Search Bar

search_bar = dbc.Row(
    [
        dbc.Col(dbc.Input(type="search", placeholder="Search")),
        dbc.Col(
            dbc.Button("Search", color="primary", className="ml-2"),
            width="auto",
        ),
    ],
    no_gutters=True,
    className="ml-auto flex-nowrap mt-3 mt-md-0",
    align="center",
)

# Nav Bar

navbar = dbc.Navbar(
    [
        html.A(
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [
                    dbc.Col(dbc.NavbarBrand("Berger-levrault", className="ml-2")),
                ],
                align="center",
                no_gutters=True,
            ),
            href="https://www.berger-levrault.com/fr/",
        ),
        dbc.NavLink("Article", active=True, href="/"),
        dbc.NavLink("Statistiques", href="/stats"),
        dbc.NavLink("Stories",href="/stories"),
        dbc.NavbarToggler(id="navbar-toggler"),
        dbc.Collapse(search_bar, id="navbar-collapse", navbar=True),
    ],
    color="dark",
    dark=True,
    fixed = "top",
    sticky = "top",
)


# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "margin-left": "2rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "2rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
    "width":"auto",
    "background-color": "#f8f9fa",
}

#Side bar

sidebar = html.Div(
    [
        html.H2("Home"),
        html.Hr(),
        html.Div([
        dbc.Col(
        [
            # -------TOUS LES DROPDOWNS + DATE PICKER ---------
            # gerer fonction du boutton voir components dash (tuto)
            dbc.Col(html.Div([html.Label('Country'),
                              dcc.Dropdown(id='comp_filt',
                    options=[
                        {'label': 'France', 'value': 'F'},
                        {'label': 'International', 'value': 'IN'}
                    ],
                    multi=True
                    )])),
            dbc.Col(html.Div([html.Label('Categorie'),
                              dcc.Dropdown(id='cat_filt',
                    options=[
                        {'label': 'Economic', 'value': 'EC'},
                        {'label': 'Ecology', 'value': 'E'}
                    ],
                    value = 'Economic',                   
                    multi=True
                    )])),
            dbc.Col(html.Div([html.Label('Date Range'), dcc.DatePickerRange(
                    id='date-picker-range',
                    start_date=dt(2016, 1, 15),
                    end_date=dt(2021, 1, 4)
                    # end_date_placeholder_text='Select a date!'
                    )])),
            dbc.Col(html.Div([html.Label('Number of article par page'), dcc.Input(
                    id = 'number',
                    type= "number",
                    value = 10,
                    min = 1,
                    max = len(df['art_title']),
                    )])),
            html.Hr(),
            dbc.Col(html.Div([html.Label('Filters'),
                              html.Br(),
                              html.Button('Refresh Filters',
                                          id='reset_button', style={"vertical-align": "middle","border-radius": "8px",
                                                                   "background-color": "white",
                                                                    "color": "black",
                                                                    "border": "2px solid #e7e7e7"})]))
            ]),
    
        ]),
        
    ],
    style=SIDEBAR_STYLE,
)


#Nav bar for pagging

nb_pages = 10

pages = dbc.NavbarSimple(
    children=[
        dbc.Col(html.Div(html.H5("Page", style = {"color":"white"}))),
        dbc.Col(html.Div([dcc.Input(
                    id = 'num_page',
                    type= "number",
                    value = 1,
                    min = 1,
                    style = {"border-radius": "8px", "color": "black", "border": "2px solid #e7e7e7",
                            "width" : "30%",
                            "background-color": "#eee"}
                    )])),
    ],
    color="dark",
)



# Content page

card = html.Div(id ='out_card')

article = html.Div(id ='out_article')

stats = html.Div(id='out_stats')

stories = html.Div(render_stories())


app.layout = html.Div(
        [
            navbar,
            html.Hr(),
            dcc.Location(id='url', refresh=False),
            html.Div(id='page-content'),
            pages
        ]
)

index_page_layout = dbc.Row(
                            [
                                dbc.Col([sidebar], width = "100%"),
                                dbc.Col(card),
                                dbc.Col(stories,width = "30%")
                            ],
                            no_gutters=True,
                        ),

stats_page_layout = dbc.Row(
                            [
                                dbc.Col([sidebar], width = "100%"),
                                dbc.Col(stats),
                            ],
                            no_gutters=True,
                        ),

article_page_layout = dbc.Col(article)

stories_page_layout = dbc.Col(stories)


# Callback

# add callback for toggling the collapse on small screens
@app.callback(
    dash.dependencies.Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
)

def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open


@app.callback(
    dash.dependencies.Output('out_card', 'children'),
    dash.dependencies.Output('num_page','max'),
    [Input('number', 'value')],
    [Input('num_page','value')],
    [Input({'type': 'tags-button','index': ALL},'n_clicks')]
)

def update_card_div(number,num_page,n):
    
    global tags_index

    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    
    filtered_df = df
    
    if "index" in changed_id:
        tags_index = int(changed_id[9])
        filtered_df = filtered_df[filtered_df["art_tag"].str.contains(top_tags[tags_index][0],case=False)]
        
    if tags_index != 11:
        filtered_df = filtered_df[filtered_df["art_tag"].str.contains(top_tags[tags_index][0],case=False)]
 
    filtered_df = filtered_df.reset_index(drop=True)
    
    length_df = len(filtered_df['art_title'])
    
    if num_page*number > length_df:
        end = length_df
        filtered_df = filtered_df[((num_page*number)-number):length_df]
    else :
        filtered_df = filtered_df[((num_page*number)-number):((num_page*number))]
        end = num_page*number
    
    cards = []
    
    
    for i in range(((num_page*number)-number),end):
        cards.append(html.Div([
            dbc.Col(html.Div([
                dbc.Card([dbc.CardImg(src=filtered_df['src_img'][i],alt="Can't display this image",top=True),
                    dbc.CardBody(
                        [
                            dbc.CardLink(html.H5(filtered_df['art_title'][i], className="card-title"), href=filtered_df['art_url'][i]),
                            html.P(filtered_df['art_content'][i][:200]),
                            dbc.Button("Learn more",id={'type': 'article-button','index': i},className="mb-3",color="primary", href ="/page-"+str(i))
                        ]
                    )],
                    className="w-100 mb-3"
                ),
            ])
            )]))
        
    return cards,(length_df//number)+1

#Content article

@app.callback(
    Output("out_article", "children"),
    [dash.dependencies.Input('url', 'pathname')]
)

def toggle_article(pathname):
    index = int(pathname.split("-")[1])

    df_article = df.iloc[index]

    article = [html.Div([
            dbc.Card([
                dbc.CardImg(src=df_article['src_img'],alt="Can't display this image",top=True),
                dbc.CardBody(
                    [
                        dbc.CardLink(html.H5(df_article['art_title'], className="card-title"), href=df_article['art_url']),
                        html.P(df_article['art_content'])
                    ]
                )],
                className="w-100 mb-3"
            ),
        ])
    ]


    return article

                 
#Statistiques
@app.callback(
    Output("out_stats", "children"),
    [dash.dependencies.Input('url', 'pathname')]
)

def render_stats(pathname):
    return html.Div("test")

#Stories

def render_stories():
    list_tags = []
    for i in range(len(top_tags)-1,-1,-1):
        list_tags.append(html.Div([
            dbc.Col(html.Div([
                dbc.Card([
                    dbc.CardBody(
                        [
                            html.H5(top_tags[i][0], className="card-title"),
                            html.H6("Number of article : " + str(top_tags[i][1]), className="card-subtitle"),
                            html.P("Petite description du tag"),
                            dbc.Button("Show this tags",id = {'type': 'tags-button','index': i}),   
                        ]
                    )],
                    className="w-100 mb-3"
                ),
            ])
            )]))
        
    
    
    return list_tags   


#Content    
    
@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])


def render_page_content(pathname):
    if pathname == "/":
        return index_page_layout
    elif pathname == "/stats":
        return stats_page_layout
    elif pathname == "/stories":
        return stories_page_layout
    else:
        return article_page_layout
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )




if __name__ == "__main__":
    app.run_server(debug=True)


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

SystemExit: 1

In [86]:
app.run_server(port=8050,host='127.0.0.1')

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run